# Network Anomaly Detection using Clustering

In [231]:
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import rbf_kernel
import random
random.seed(42)
import warnings
warnings.filterwarnings('ignore')
from sklearn.cluster import KMeans
import numpy as np
import math
from scipy.optimize import linear_sum_assignment
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cluster import SpectralClustering
import pandas as pd

# 1. Importing Data and Understanding Format

In [19]:
from ipynb.fs.full.data_preprocessing import preprocess_data_10, preprocess_data

In [20]:
data_k_means, labels_kmeans = preprocess_data_10()
data_spectral, labels_spectral = preprocess_data()

# 2.  Clustering Using K-Means and Normalized Cut (Your implementation)


### K-Means algorithm

In [42]:
#It takes two attrs k number of centroids and the whole data set number of samples x features

def kMeans_implemented(k,data):
    centroids=[]
    num_points=data.shape[0]
    num_features=data.shape[1]
    
    #Appending random points to be our centroids according to the number of ks
    for i in range(k):
        centroids.append(data[random.randint(0, num_points)])
    clusters={}
    t=0
    while(True):
        labels=[]
        #Initialize empty clusters
        for i in range (k):
            clusters[i]=[]
            
        #Classify the points according to the closest centroid
        for i in range(num_points):
            distances=[]
            for j in range(k):
                distances.append(np.linalg.norm(data[i]-centroids[j]))
            clusters[distances.index(min(distances))].append(data[i])
            labels.append(distances.index(min(distances)))
        new_centroids=np.zeros((k,num_features))
        
        #Measuring the new centroids
        for i in range(k):
            new_centroids[i]=np.mean(clusters[i],axis=0)
        if(centroids==new_centroids).all():
            break
        else:
            centroids=new_centroids
    return labels

### Spectral Clustering algorithm

In [43]:
from sklearn.cluster import KMeans
def spectral_clustering(A,k):
        
    #--------------computing the degree matrix-------------
    d = np.diag(np.sum(A, axis=1))

    #--------------------computing L-----------------------
    L = d-A

    #---------------------computing La---------------------
    #computing the inverse of the dgree matrix
    inv_degree = np.linalg.inv(d)
    La = np.dot(inv_degree, L)

    #---computing the eigenValues and eigenVectors of La---
    e_val, evec = np.linalg.eig(La)

    #----------sorting the eigenValues ascending----------- 
    idx = np.argsort(eval)
    e_val = e_val[idx]

    #---sorting the eigenVectors according to their corresponding eigenValues---
    evec = evec[:, idx]

    #--slicing the eigenVectors to the desired number of clusters--
    evec_new = evec[:, :k]

    #-------------normalizing the eigenVectors--------------
    system = evec.real / np.sqrt(np.linalg.norm(evec.real))

    kmeans = KMeans(n_clusters=k)
    system_labels = kmeans.fit_predict(system)


    return system, system_labels

## DBSCAN algorithm

In [234]:
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances

def dbscan(X, eps=0.5, min_samples=10):
    """
    DBSCAN clustering algorithm implementation from scratch.
    
    Arguments:
    X -- numpy array of shape (n_samples, n_features), representing the dataset to be clustered
    eps -- float, the maximum distance between two points in the same neighborhood (default=0.5)
    min_samples -- int, the minimum number of points required to form a dense region (default=5)
    
    Returns:
    labels -- numpy array of shape (n_samples,), representing the cluster assignments of each data point
    """
    
    # initialize the list of core points and the array of cluster labels
    core_points = []
    labels = np.full(X.shape[0], -1)
    cluster_id = 0
    
    # compute the pairwise distances between data points
    distances = pairwise_distances(X)
    
    # identify the core points and their neighborhoods
    for i in range(X.shape[0]):
        neighborhood = np.where(distances[i] <= eps)[0]
        if neighborhood.shape[0] >= min_samples:
            core_points.append(i)
            labels[i] = cluster_id
            for j in neighborhood:
                if labels[j] == -1:
                    labels[j] = cluster_id
                    
            # grow the cluster
            while True:
                old_size = np.sum(labels == cluster_id)
                for j in core_points:
                    neighborhood = np.where(distances[j] <= eps)[0]
                    for k in neighborhood:
                        if labels[k] == -1:
                            labels[k] = cluster_id
                            core_points.append(k)
                new_size = np.sum(labels == cluster_id)
                if old_size == new_size:
                    break
            
            # move to the next cluster
            cluster_id += 1
    
    return labels


In [216]:
# Splitting data to use it in spectral clustering
X_train, X_test, y_train, y_test = train_test_split(data_spectral, labels_spectral, test_size=0.995, train_size=0.005,stratify=labels_spectral,random_state=42)

In [217]:
sim_matrix=rbf_kernel(X_train)

In [218]:
system,labels=spectral_clustering(sim_matrix,23)

In [235]:
labels = dbscan(X_train)

In [236]:
labels = pd.DataFrame(labels)
labels.value_counts()

-1       1620
 3146      10
 1031       7
 705        7
 613        6
         ... 
 1231       1
 1232       1
 1233       1
 1234       1
 3561       1
Length: 3563, dtype: int64

# Evaluation

In [181]:
# map labels resulting in k-means to true labels in able to do predictions
def map_and_change(y_train, labels):
    mapping = {}
    labels = np.array(list(labels))
    for i in np.unique(labels):
        binary = [int(x) for x in labels == i]
        mapping[i] = np.bincount([value for value, flag in zip(y_train, binary) if flag == 1]).argmax()

    # Map the cluster labels to the true class labels
    mapped_labels = np.array([mapping[label] for label in labels])

    # Print the mapped labels
    print(mapping)
    return mapping, mapped_labels

In [180]:
def map_and_change_test(mapping, labels):
    mapped_labels = np.array([mapping[label] for label in labels])
    return mapped_labels

In [150]:
new_labels = map_and_change(y_train,labels)

{0: 9, 1: 20, 2: 20, 3: 20, 4: 20, 5: 20, 6: 14, 7: 11, 8: 11, 9: 11, 10: 11, 11: 21, 12: 11, 13: 11, 14: 11, 15: 11, 16: 11, 17: 9, 18: 18, 19: 11, 20: 11, 21: 11, 22: 11}


In [80]:
def evaluation(data,contingency_matrix):
    
    n_total = data.shape[0]
    gt_classes=contingency_matrix.shape[0]
    predicted_classes=contingency_matrix.shape[1]
    TP, TN, FP, FN = 0, 0, 0, 0
    # True Positive 
    for i in range(gt_classes):
        for j in range(predicted_classes):
            if contingency_matrix[i][j] != 1 and contingency_matrix[i][j] != 0:
                TP += math.comb(int(contingency_matrix[i][j]),2)

    # True Negative 
    for i in range(gt_classes):
        for j in range(predicted_classes):
            if i != j:
                for k in range(predicted_classes):
                    temp = contingency_matrix[k,i]*(np.sum(contingency_matrix[:,j]) - contingency_matrix[k,j])
                    TN += temp
    TN = TN/2

    # False Positive 
    for i in range(gt_classes):
        for j in range(predicted_classes):
            temp = contingency_matrix[j,i]*(np.sum(contingency_matrix[:,i])-contingency_matrix[j,i])/2
            FP += temp

    # False Negative 
    for i in range(gt_classes):
        for j in range(predicted_classes):
            if i != j:
                for k in range(predicted_classes):
                    temp = contingency_matrix[k,i]*(contingency_matrix[k,j])
                    FN += temp
    FN /= 2

    # Jaccard Index
    jacc = TP / (TP + FN + FP)

    # Rand Index
    rand = (TP + TN)/ (TP + FN + FP + TN)
    print('---------Confusion Matrix----------')
    print(f"Rand Index: {rand}")
    
    print(f"Jaccard Index: {jacc}")
    print(f'TP= {TP},TN= {TN},FN= {FN},FP= {FP}')
    
    ht_c = 0
    for i in range(gt_classes):
        cluster_elem = np.sum(contingency_matrix[:,i])
        for j in range(predicted_classes):  
            temp = contingency_matrix[j][i]/cluster_elem
            if temp != 0:
                ht_c += temp*math.log(temp,2)*cluster_elem/n_total
    ht_c = -1*ht_c
    print('---------Conditional Entropy----------')
    print(f"Conditional Entropy: {ht_c}")
    
    print('----------------Purity----------------')
    purity=0
    purities=[]
    recalls=[]
    for i in range(predicted_classes):
        cluster_sum=np.sum(contingency_matrix[:,i])
        class_max=np.max(contingency_matrix[:,i])
        a=contingency_matrix[:,i]
        max_index=a.argmax()
        purities.append(class_max/cluster_sum)
        recalls.append(class_max/np.sum(contingency_matrix[max_index,:]))
        purity+=(class_max/cluster_sum) * (cluster_sum/n_total)
    #purity = np.sum(np.max(contingency_matrix, axis =0))/np.sum(contingency_matrix)
    print(f"Purity: {purity}")
    #print("Purities",purities)
    
    print('--------------F-measure---------------')
    # a row for each cluster, and columns are precision, recall and F-measure respectively
    
    f_measure=0
    for i in range(predicted_classes):
        f_measure+=(2*purities[i]*recalls[i])/(purities[i]+recalls[i])
    f_measure=f_measure/predicted_classes
    print(f"F: {f_measure}")
    
    print('--------------Max matching---------------')
    row_ind, col_ind = linear_sum_assignment(contingency_matrix, maximize=True)
    contingency_reordered = contingency_matrix[row_ind][:, col_ind]
    #print(contingency_reordered)
    max_match = np.sum(np.diag(contingency_reordered))/np.sum(contingency_matrix)
    print(f"Max Matching: {max_match}")

In [62]:
num_classes = 23
num_elements = len(labels)
contingency_matrix = np.zeros((num_classes,num_classes))
for i in range(num_elements):
    contingency_matrix[y_train[i],labels[i]] += 1

In [63]:
evaluation(X_train,contingency_matrix)

---------Confusion Matrix----------
Rand Index: 0.4672537036309752
Jaccard Index: 0.16607228898941917
TP= 1531700,TN= 5214159.0,FN= 7453691.0,FP= 237701.0
---------Conditional Entropy----------
Conditional Entropy: 0.389318100386387
----------------Purity----------------
Purity: 0.8823967249720878
--------------F-measure---------------
F: 0.27426062013312574
--------------Max matching---------------
Max Matching: 0.2967994045403796


# Testing K-means using Test Data set and Mapping Clusters to Classes

In [173]:
X_train, X_test, y_train, y_test = train_test_split(data_spectral, labels_spectral, test_size=0.995, train_size=0.005,stratify=labels_spectral,random_state=42)

In [183]:
kmeans = KMeans(n_clusters=23, random_state=42)

In [184]:
kmeans.fit(X_train)

KMeans(n_clusters=23, random_state=42)

In [185]:
train_labels = kmeans.labels_

In [186]:
mapping, train_labels = map_and_change(y_train, train_labels)

{0: 11, 1: 9, 2: 9, 3: 11, 4: 11, 5: 20, 6: 11, 7: 11, 8: 11, 9: 11, 10: 11, 11: 11, 12: 11, 13: 11, 14: 11, 15: 11, 16: 11, 17: 17, 18: 5, 19: 11, 20: 11, 21: 11, 22: 11}


In [178]:
test_labels = kmeans.predict(X_test)

In [187]:
test_labels = map_and_change_test(mapping,test_labels)

In [188]:
accuracy = accuracy_score(test_labels, y_test)

In [190]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.9860987754506749
